In [1]:
import time
import numpy as np

import mss
import cv2
from PIL import Image

import matplotlib.pyplot as plt

from xpc3 import *
from xpc3_helper_sm import *

In [2]:
client = XPlaneConnect()
reset(client, dtpInit=270.0)

# Useful Functions

In [3]:
def get_pixel_line(A, B, sh, sw):
    # Convert to homogeneous and get Plucker
    Ah = np.append(np.array(A), 1.0)
    Bh = np.append(np.array(B), 1.0)
    L = np.outer(Ah, Bh) - np.outer(Bh, Ah)

    # Get projection matrix
    mv = np.reshape(client.getDREF("sim/graphics/view/world_matrix"), (4, 4)).T
    proj = np.reshape(client.getDREF(
        "sim/graphics/view/projection_matrix_3d"), (4, 4)).T
    proj_3 = np.zeros((3, 4))
    proj_3[:2, :] = proj[:2, :]
    proj_3[2, :] = proj[3, :]
    screen_mat = np.array([[0.5 * sw, 0.0, 0.5 * sw],
                           [0.0, -0.5 * sh, 0.5 * sh],
                           [0.0, 0.0, 1.0]])
    P = screen_mat @ proj_3 @ mv

    lx = P @ L @ P.T
    return np.array([lx[2, 1], lx[0, 2], lx[1, 0]])


def get_plane(A, B, C):
    v1 = A - B
    v2 = C - A
    n = np.cross(v1, v2)
    k = -np.dot(A, n)
    return np.array([n[0], n[1], n[2], k])


def get_plane_int(p1, p2):
    p1_normal = p1[:3]
    p2_normal = p2[:3]

    p3_normal = np.cross(p1_normal, p2_normal)
    det = np.sum(np.square(p3_normal))

    r_point = (np.cross(p3_normal, p2_normal) *
               p1[3] + np.cross(p1_normal, p3_normal) * p2[3]) / det
    r_normal = p3_normal

    return r_point, r_normal


def get_pixels(line, sw):
    a, b, c = line
    xs = np.arange(0, sw, 1)
    ys = [-(a / b) * x + (-c / b) for x in xs]
    return xs, ys


def plot_line_in_image(ss, A, B):
    sh, sw, _ = ss.shape
    l = get_pixel_line(A, B, sh, sw)
    xs, ys = get_pixels(l, sw)
    plt.figure(figsize=(10, 6))
    plt.imshow(ss)
    plt.plot(xs, ys, c='lime', linewidth=2.0)
    plt.axis('off')


# Get Anchor Values

In [4]:
screen_shot = mss.mss()
ss = cv2.cvtColor(np.array(screen_shot.grab(
    screen_shot.monitors[2])), cv2.COLOR_BGRA2BGR)[:, :, ::-1]
sh, sw, _ = ss.shape

In [5]:
H0 = np.reshape(client.getDREF("sim/graphics/view/world_matrix"), (4, 4)).T
proj = np.reshape(client.getDREF(
    "sim/graphics/view/projection_matrix_3d"), (4, 4)).T
proj_3 = np.zeros((3, 4))
proj_3[:2, :] = proj[:2, :]
proj_3[2, :] = proj[3, :]
screen_mat = np.array([[0.5 * sw, 0.0, 0.5 * sw],
                       [0.0, -0.5 * sh, 0.5 * sh],
                       [0.0, 0.0, 1.0]])
K0 = screen_mat @ proj_3

print("H0: ", H0)
print("K0: ", K0)

H0:  [[-9.14254069e-01  0.00000000e+00  4.05141264e-01  1.38986855e+04]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.81350555e+02]
 [-4.05141264e-01  0.00000000e+00 -9.14254069e-01  5.67687930e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
K0:  [[ 1.14408348e+03  0.00000000e+00 -9.60000000e+02  0.00000000e+00]
 [ 0.00000000e+00 -1.14408334e+03 -5.40000000e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.00000000e+00  0.00000000e+00]]


In [7]:
l1 = np.array([35929.9796875, -190.48041382, 46803.98203125])
l2 = np.array([35753.2046875, -185.55480957, 46404.71953125])
r1 = np.array([35950.821875, -190.52763977, 46794.65])
r2 = np.array([35773.7703125, -185.58735657, 46395.5515625])

In [23]:
r1c = H0 @ np.append(r1, 1.0)
r2c = H0 @ np.append(r2, 1.0)
l1c = H0 @ np.append(l1, 1.0)
l2c = H0 @ np.append(l2, 1.0)
c1c = (l1c[:3] + r1c[:3]) / 2

In [17]:
ground_plane = get_plane(np.array(r1c[:3]), np.array(r2c[:3]), np.array(l1c[:3]))

# Try with new reference frame

In [18]:
def get_corresponding_plane(l):
    return K0.T @ l


def get_3d_line(l, ground_plane):
    edge_plane = get_corresponding_plane(l)
    r_point, r_normal = get_plane_int(edge_plane, ground_plane)
    return r_point, r_normal

In [21]:
setHomeState(client, 0, 270, 0)
time.sleep(0.2)
line_right = get_pixel_line(r1, r2, sh, sw)
line_right_a, line_right_u = get_3d_line(line_right, ground_plane)
print((r1c[:3] - line_right_a) / line_right_u)
line_left = get_pixel_line(l1, l2, sh, sw)
line_left_a, line_left_u = get_3d_line(line_left, ground_plane)
print((l1c[:3] - line_left_a) / line_left_u)

[3.89413289e-11 3.89413289e-11 3.89413289e-11]
[3.86602918e-11 3.86602917e-11 3.86602917e-11]


In [22]:
def get_distance(p1, l1, p2):
    n = l1 / np.linalg.norm(l1)
    m = np.cross(p1, n)

    d = np.linalg.norm(np.cross(p2, n) - m)

    return d

In [25]:
print(get_distance(line_right_a, line_right_u, c1c))
print(get_distance(line_left_a, line_left_u, c1c))

11.417965206664903
11.417926637334881


In [31]:
setHomeState(client, 8, 320, 0)
time.sleep(0.2)
line_right = get_pixel_line(r1, r2, sh, sw)
line_right_a, line_right_u = get_3d_line(line_right, ground_plane)
# print((r1 - line_right_a) / line_right_u)
line_left = get_pixel_line(l1, l2, sh, sw)
line_left_a, line_left_u = get_3d_line(line_left, ground_plane)
# print((l1 - line_left_a) / line_left_u)

11.417965507841265 - get_distance(line_right_a, line_right_u, c1c)

7.94325378311388

# Rotation

In [45]:
def get_rot_x(theta):
    return np.array([[1.0, 0.0, 0.0],
                     [0.0, np.cos(theta), -np.sin(theta)],
                     [0.0, np.sin(theta), np.cos(theta)]])


def get_rot_y(theta):
    return np.array([[np.cos(theta), 0.0, np.sin(theta)],
                     [0.0, 1.0, 0.0],
                     [-np.sin(theta), 0.0, np.cos(theta)]])


def get_rot_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0.0],
                     [np.sin(theta), np.cos(theta), 0.0],
                     [0.0, 1.0, 0.0]])


In [53]:
setHomeState(client, 0, 270, 0)
time.sleep(0.2)
line_right0 = get_pixel_line(r1, r2, sh, sw)
line_right0_a, line_right0_u = get_3d_line(line_right0, ground_plane)
print((r1 - line_right0_a) / line_right0_u)
line_left0 = get_pixel_line(l1, l2, sh, sw)
line_left0_a, line_left0_u = get_3d_line(line_left0, ground_plane)
print((l1 - line_left_a) / line_left_u)

[-5.90792060e-06  1.11761471e-09 -3.14939294e-09]
[ 1.38538904e-08  1.15439004e-09 -3.14706415e-09]


In [54]:
client.getDREF("sim/flightmodel/position/local_y")[0]


-182.10951232910156

In [107]:
setHomeState(client, 8, 270, 10)
time.sleep(0.2)
line_right = get_pixel_line(r1, r2, sh, sw)
line_right_a, line_right_u = get_3d_line(line_right, ground_plane)
print((r1 - line_right_a) / line_right_u)
line_left = get_pixel_line(l1, l2, sh, sw)
line_left_a, line_left_u = get_3d_line(line_left, ground_plane)
print((l1 - line_left_a) / line_left_u)

[ 0.00000001  0.         -0.        ]
[ 0.00000001  0.         -0.        ]


In [108]:
Ry = get_rot_y(np.radians(10))
line_right_arot, line_right_urot = Ry @ line_right_a, Ry @ line_right_u
11.417965507841265 - get_distance(line_right_arot, line_right_urot, c1c)

8.472877205564863

In [39]:
line_right0_u / np.linalg.norm(line_right0_u)

array([-4.09645300e-04, -1.13144138e-02, -9.99935906e-01])

In [58]:
R = get_rot_y(-10 * np.pi / 180)
urot = R @ line_right_u
urot / line_right0_u

array([-832.24107831,    0.94402912,    0.93152369])

In [42]:
K0

array([[ 1.14408348e+03,  0.00000000e+00, -9.60000000e+02,
         0.00000000e+00],
       [ 0.00000000e+00, -1.14408334e+03, -5.40000000e+02,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -1.00000000e+00,
         0.00000000e+00]])

In [66]:
np.set_printoptions(suppress=True)
ground_plane / ground_plane[3]


array([-0.00080976,  0.38348469, -0.00433885,  1.        ])

In [61]:
l1c

array([  11.77985644,   -9.1298584 , -578.65545737,    1.        ])

In [62]:
r1c

array([ -11.05598923,   -9.17708435, -578.56764002,    1.        ])

In [63]:
r2c

array([ -10.87712325,   -4.23680115, -141.95937561,    1.        ])

In [67]:
u1, u2 = line_right0_u, line_right_u

In [71]:
np.degrees(np.arccos(np.dot(u1, u2) / (np.linalg.norm(u1) * np.linalg.norm(u2))))

10.125952188121646

In [72]:
def rotate_around_vec(vec, pt, theta):
    u = vec / np.linalg.norm(vec)
    W = np.array([[0, -u[2], u[1]],
                  [u[2], 0, -u[0]],
                  [-u[1], u[0], 0.0]])
    R = np.eye(3) + np.sin(theta) * W + (2 * (np.sin(theta / 2))**2) * (W @ W)
    return R @ pt

In [75]:
u2prime = rotate_around_vec(ground_plane[:3], u1, np.radians(-10))

In [78]:
def get_rot_a_to_b(ain, bin):
    a = ain / np.linalg.norm(ain)
    b = bin / np.linalg.norm(bin)

    v = np.cross(a, b)
    s = np.linalg.norm(v)
    c = np.dot(a, b)
    vx = np.array([[0.0, -v[2], v[1]],
                   [v[2], 0.0, -v[0]],
                   [-v[1], v[0], 0.0]])
    
    return np.eye(3) + vx + vx @ vx * ((1 - c) / (s**2))

In [80]:
u1, u2 = line_right0_u / np.linalg.norm(line_right0_u), line_right_u / np.linalg.norm(line_right_u)

In [90]:
R = get_rot_a_to_b(u2, u1)
R

array([[ 0.98442372,  0.00195617,  0.17580137],
       [-0.00202195,  0.99999794,  0.000195  ],
       [-0.17580062, -0.00054743,  0.98442564]])

In [91]:
Ry = get_rot_y(np.radians(10))
Ry

array([[ 0.98480775,  0.        ,  0.17364818],
       [ 0.        ,  1.        ,  0.        ],
       [-0.17364818,  0.        ,  0.98480775]])

In [94]:
Ry @ u2

array([ 0.00179846, -0.0107678 , -0.99994041])

In [95]:
u1

array([-0.00040965, -0.01131441, -0.99993591])

# Try out full calculation pipeline

In [111]:
# Things needed
c1c, u1, ground_plane, K0

(array([   0.3619336 ,   -9.15347138, -578.61154869]),
 array([-0.00040965, -0.01131441, -0.99993591]),
 array([    21.05308266,  -9970.33465342,    112.80709762, -25999.30320831]),
 array([[ 1144.08348083,     0.        ,  -960.        ,     0.        ],
        [    0.        , -1144.08333778,  -540.        ,     0.        ],
        [    0.        ,     0.        ,    -1.        ,     0.        ]]))

In [112]:
def get_corresponding_plane(l, K0):
    return K0.T @ l


def get_3d_line(l, K0, ground_plane):
    edge_plane = get_corresponding_plane(l, K0)
    r_point, r_normal = get_plane_int(edge_plane, ground_plane)
    return r_point, r_normal

In [203]:
def get_state(pixel_line, K0, ground_plane, u1, c1):
    p2, u2 = get_3d_line(pixel_line, K0, ground_plane)

    # Determine heading
    cos_heading = np.dot(u1, u2) / (np.linalg.norm(u1) * np.linalg.norm(u2))
    if cos_heading > 1.0:
        cos_heading = 1.0
    heading = np.arccos(cos_heading) # degrees

    # Unrotate
    Ry = get_rot_y(heading)
    p2rot, u2rot = Ry @ p2, Ry @ u2
    if np.linalg.norm(np.cross(u1, u2rot / np.linalg.norm(u2rot))) > 1e-2:
        # Rotated the wrong way
        heading = -heading
        Ry = get_rot_y(heading)
        p2rot, u2rot = Ry @ p2, Ry @ u2

    # Determine crosstrack
    crosstrack = 11.417965507841265 - get_distance(p2rot, u2rot, c1)

    return crosstrack, np.degrees(heading)


In [221]:
setHomeState(client, -2.0, 270, 5)
time.sleep(0.2)
line_right = get_pixel_line(r1, r2, sh, sw)
line_left = get_pixel_line(l1, l2, sh, sw)

In [222]:
get_state(line_left, K0, ground_plane, u1, c1c)

(2.458862834820529, 5.000808949443581)